## Ex 2: Realistic Gravity

In [1]:
import matplotlib.pyplot as plt
from space_base import GravBody, Probe
from numpy import max, argmax

# Constants
G = 6.67e-11  # Gravitational constant
earth = GravBody.earth()  # Earth as an object with mass and radius
gravity = 9.81  # simple gravity

# Initial Conditions
x0 = 0  # start position
vx0 = 850  # start vertical speed
t_num = 1400  # number of steps in trajectory

First, we start by running the previous simulation with uniform gravity and then do the same simulation but with a more realistic approach.

In [2]:
def projectile_uniform_gravity(t, posvel):
    current_gravity = gravity
    return posvel[1], -current_gravity

t_final = (2*vx0) / gravity  # time of trajectory given
# Running Solver
probe = Probe(projectile_uniform_gravity, t_final, t_num, x0=x0, vx0=vx0, event=0)  # probe as an object
t, posvel = probe.odesolve()  # solve the differential equations
max_height_uniform_gravity = max(posvel, axis=0)[0]

Current gravity can be compute as follows:
$$g=\frac{GM}{(R+z)^2}$$
Where, $M$ is the earth mass, $R$ its radius and $z$ is the current altitude.
Of course, we also need to update the energy as it is now define as:
$$E=\frac{1}{2}m*v^2 - \frac{GMm}{R+z}$$ 

In [3]:
def projectile_with_gravity(t, posvel):
    current_gravity = G * earth.mass / (earth.radius + posvel[0])**2
    return posvel[1], -current_gravity

t_final_temp = 200
# Running Solver
probe = Probe(projectile_with_gravity, t_final_temp, t_num, x0=x0, vx0=vx0, event=0)  # probe as an object
t, posvel = probe.odesolve()  # solve the differential equations

# Solver Results
max_height_realistic_gravity = max(posvel, axis=0)[0]
t_final_index = argmax(posvel[1:, 0]<=0)
t_end = t_final_index if t_final_index < len(t) else len(t) - 1   # last valid value of array (corresponds to index of t_final)
t_final_realistic_gravity = t[t_end]

With the new way of computing gravity we found that the maximum altitude if higher by:

In [4]:
max_height_realistic_gravity - max_height_uniform_gravity

198.99292610568227

As the trajectory is modified we can assume that time of flight will also be longer.

In [5]:
t_final_realistic_gravity - t_final

1.2606937094284092

We can now check if the energy is well conserved.

In [6]:
in_energy = 0.5 * probe.mass * posvel[0][1] ** 2 - G * earth.mass * probe.mass  / (earth.radius + posvel[0][0])  # initial energy
fin_energy = 0.5 * probe.mass * posvel[t_end][1] ** 2 - G * earth.mass * probe.mass  / (earth.radius + posvel[t_end][0])  # final energy
accuracy = (fin_energy - in_energy) / in_energy  # accuracy of solver
accuracy

1.4741062648021833e-10

We can also expressed the accurary as a percentage that is close to 100% if the energy at the beginning and the end of the simulation are the same.

In [7]:
accuracy = 100 * in_energy / fin_energy  # accuracy of solver
accuracy

99.99999998525892

Or, in contrary, make an error computation in percentage (we will then want to keep this percentage as low as possible):

In [8]:
error_percentage = 100 * abs((fin_energy - in_energy) / in_energy)
error_percentage

1.4741062648021833e-08